In [35]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd

In [34]:
# Read in Exoplanet Database csv
exo_df = pd.read_csv("pydata/exoplanet_database.csv", skiprows=[1])
print(list(exo_df.columns))

['STAR', 'COMP', 'ORBREF', 'ORBURL', 'RA', 'FIRSTREF', 'FIRSTURL', 'DEC', 'MSTAR', 'MSTARUPPER', 'MSTARLOWER', 'UMSTAR', 'RSTAR', 'RSTARUPPER', 'RSTARLOWER', 'URSTAR', 'DIST', 'DISTUPPER', 'DISTLOWER', 'UDIST', 'TEFF', 'TEFFUPPER', 'TEFFLOWER', 'UTEFF', 'LOGG', 'LOGGUPPER', 'LOGGLOWER', 'ULOGG', 'FE', 'FEUPPER', 'FELOWER', 'UFE']


In [144]:
# Collect list of stellar parameter files
stellar_param_files = glob.glob("pydata/sumout_files/*")

In [143]:
# Generate Stellar Parameter DataFrame
stellar_param_dict = {}
files = ["pydata/sumout_files/11Com_2012.m.sumout"]
for file in stellar_param_files:
    with open(file,'r') as f:
        for i, line in enumerate(f):
            if i == 0:
                star = f.readline().split(" ")[0].split("_")[0]
                #print(star)
                params = [elem for elem in f.readline().split(" ") if elem != '']
                #print(params)
                metallicity = params[-2]
                if '-' in metallicity:
                    metallicity = metallicity[4:]
                file_dict = {'Teff': float(params[2]), 'Log_g':float(params[6]), 'M/H':float(metallicity)}
                #print(file_dict)
                stellar_param_dict[star] = file_dict
                
param_df = pd.DataFrame.from_dict(stellar_param_dict, orient='index')

In [54]:
subset = [s for s in list(exo_df['STAR']) if "Com" in s]
exo_df[exo_df['STAR'] == '11 Com']

,STAR,COMP,ORBREF,ORBURL,RA,FIRSTREF,FIRSTURL,DEC,MSTAR,MSTARUPPER,...,TEFFLOWER,UTEFF,LOGG,LOGGUPPER,LOGGLOWER,ULOGG,FE,FEUPPER,FELOWER,UFE
2707,11 Com,b,Liu 2008,http://adsabs.harvard.edu/abs/2008ApJ...672..553L,12.345304,Liu 2008,http://adsabs.harvard.edu/abs/2008ApJ...672..553L,17.792653,2.04,0.29,...,79.0,79.0,2.61,0.13,0.13,0.13,-0.34,0.06,0.06,0.06
